In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
data = pd.read_csv('../2004-2018_ALERTS.csv', sep=';', 
                   header=0, index_col = 0)
data['DATE_CASE'] =pd.to_datetime(data.DATE_CASE, yearfirst=True, format = '%Y/%m/%d')

In [3]:
columns = list(data)
bad_columns = ['NUMBER','REF','SUBJET']
q = 0.95 # % of confidence
p_dict = {}

In [4]:
for b in bad_columns:
    columns.remove(b)
print(columns)
for a in columns:
    for i in columns:
        equal = False
        if a == i:
            equal = True
        if equal == False :
            key = a + " con " + i
            selected_data = pd.DataFrame(data.groupby([a,i]).size()).unstack(level=-1)[0]
            selected_data.loc[i + '_Totals']= selected_data.sum()
            selected_data[a + '_Totals'] = selected_data.sum(axis=1)
            selected_data = selected_data.fillna(0)
            selected_wo_totals = selected_data.iloc[0:len(selected_data.index)-1,0:len(selected_data.columns)-1]
            x = len(selected_wo_totals.index)
            y = len(selected_wo_totals.columns)
            expected =  np.outer(selected_data[a + '_Totals'][0:x], selected_data.loc[i + '_Totals'][0:y]) / selected_data.iloc[x,y]
            expected = pd.DataFrame(expected)
            expected.columns = list(selected_wo_totals)
            expected.index = selected_wo_totals.index.tolist()
            chi_squared_stat_table = (((selected_wo_totals-expected)**2)/expected)
            chi_squared_stat = (((selected_wo_totals-expected)**2)/expected).sum().sum()
            dof = (x-1)*(y-1)
            crit = stats.chi2.ppf(q = q, df = dof)   # Find the critical value for q confidence*
            p_value = 1 - stats.chi2.cdf(x=chi_squared_stat, df=dof)
            conti = stats.chi2_contingency(observed=selected_wo_totals)
            p_dict[key] = conti[1]

['CLASSIF', 'DATE_CASE', 'NOT_COUNTRY', 'PROD_CAT', 'TYPE', 'RISK_DECISION', 'ACTION_TAKEN', 'DISTRIBUTION_STAT', 'PRODUCT', 'HAZARDS', 'HAZARDS_CAT', 'COUNT_ORIGEN', 'COUNT_DESTIN', 'COUNT_CONCERN']


In [5]:
df_p = pd.DataFrame(list(p_dict.items()))
df_p.sort_values(by=[1])

,0,1
0,CLASSIF con DATE_CASE,0.000000e+00
113,PRODUCT con HAZARDS_CAT,0.000000e+00
114,PRODUCT con COUNT_ORIGEN,0.000000e+00
115,PRODUCT con COUNT_DESTIN,0.000000e+00
117,HAZARDS con CLASSIF,0.000000e+00
...,...,...
53,TYPE con DATE_CASE,1.647631e-126
161,COUNT_DESTIN con RISK_DECISION,1.915048e-86
76,RISK_DECISION con COUNT_DESTIN,1.915048e-86
116,PRODUCT con COUNT_CONCERN,1.000000e+00
